In [1]:
import json
import os.path as op
from pprint import pprint

assert op.exists('functions.json')
assert op.exists('expressions.json')

### Generate VEX functions completions

Write completions into a `functions.sublime-completions` file.

In [2]:
with open('functions.json') as f:
    functions = json.load(f)

comps = {
    'scope': 'source.vex -source.hscript -string',
    'completions': []
}
unique_triggers = set()  # Keep track of existing completions.

for f in sorted(functions, key=lambda x: x.lower()):
    for sig in functions[f]['sigs']:
        args = sig['argnames'][:]

        # Add vararg ellipsis.
        if 'variadic' in sig:
            if args:
                args[-1] += ', ...'
            else:
                args = ['...']

        trigger = '%s(%s)' %(f, ', '.join(args))
        
        # Completion exists.
        if trigger in unique_triggers:
            continue
        # Completion not exists. Make a new one.
        unique_triggers |= {trigger}

        # Build contents.
        args = []
        counter = 1
        for i, arg in enumerate(sig['argnames'], 1):
            args.append('${%d:%s}' %(i, arg))
            counter = i + 1
        # Add vararg ellipsis.
        if 'variadic' in sig:
            if args:
                args[-1] += '${%d:, ...}' %counter
            else:
                args = ['${%d:...}' %counter]

        contents = '%s(%s)' %(f, ', '.join(args))
        comps['completions'].append({'trigger': trigger,
                                     'contents': contents})

print('Generated %d completions.' % len(unique_triggers))

with open('functions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 1307 completions.


### Generate HScript Expressions completions

Write completions into a `expressions.sublime-completions` file.

In [3]:
with open('expressions.json') as f:
    expressions = json.load(f)

comps = {
    'scope': 'source.hscript',
    'completions': []
}

for f in sorted(expressions, key=lambda x: x.lower()):
    name = f
    if 'args' in expressions[f]:
        names = expressions[f]['argnames']
        args = [' '.join(a) for a in zip(expressions[f]['args'], names)]
    else:
        names = []
        args = []

    subs = []
    for i, arg in enumerate(args, 1):
        subs.append('${{{}:{}}}'.format(i, arg))

    trigger = '{}({})'.format(f, ', '.join(names))
    contents = '{}({})'.format(f, ', '.join(subs))
    comps['completions'].append({'trigger': trigger,
                                 'contents': contents})

print('Generated %d completions.' % len(comps['completions']))

with open('expressions.sublime-completions', 'w') as f:
    json.dump(comps, f, indent=4)

Generated 390 completions.


### Generate helpcards file

Assumed HTML aready generated.
    
#### Input structure:

`functions` and `expressions`
```yaml
foo:
    help: <html>foo<html>
```

#### Output structure:

`helpcards`
```yaml
hscript:
    foo: <html>foo<html>
vex:
    foo: <html>foo<html>
```

In [4]:
# Format into output structure.
helpcards = {
    'hscript': {f : expressions[f]['help']
                for f in expressions
                if 'help' in expressions[f]},
    'vex': {f : functions[f]['help']
            for f in functions
            if 'help' in functions[f]}
}

print('Written %d helpcards for HScript Expressions.' % len(helpcards['hscript']))
print('Written %d helpcards for VEX functions.' % len(helpcards['vex']))

with open('helpcards.json', 'w') as f:
    json.dump(helpcards, f, indent=4, sort_keys=True)

Written 388 helpcards for HScript Expressions.
Written 702 helpcards for VEX functions.
